<a href="https://colab.research.google.com/github/RaginiMeshram/Tensorflow/blob/master/DR_training_30_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import h5py
import numpy as np
from PIL import Image
from matplotlib import  pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
os.getcwd()

'/content'

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [7]:
!pwd

/content/drive/My Drive/Colab Notebooks


In [10]:
os.listdir('/content/drive/My Drive/Colab Notebooks/')

['Untitled',
 'DR_training_30_Nov.ipynb',
 'More_DR_patches_with_grading.zip',
 'More_DR_patches_with_grading']

In [0]:

# This is for extracting the files
import zipfile
path_to_zip_file = 'More_DR_patches_with_grading.zip'
directory_to_extract_to = '/content/drive/My Drive/Colab Notebooks/'
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [0]:
PREPROCESSED_DATA_FOLDER= "/content/drive/My Drive/Colab Notebooks/More_DR_patches_with_grading"

In [12]:
import glob
import ntpath
import cv2
import numpy as np

# CNN MODEL
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
images = []
classes = []

imgNone = 0
imgCnt = 0

for classFolderPath in os.listdir(PREPROCESSED_DATA_FOLDER):
    
    pattern = os.path.join(PREPROCESSED_DATA_FOLDER, classFolderPath, "*.tif")
    
    for imagePath in glob.glob( pattern ):

        img = cv2.imread(imagePath, cv2.IMREAD_COLOR)
        
        # if the file contain any missig value ignore it
        if img is None:
            imgNone += 1
            continue
            
        imgCnt += 1
        
        images.append( img )
        classes.append(classFolderPath)

In [0]:
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(224,224), interpolation = cv2.INTER_CUBIC)
  return res
#resized=[]
#for image in images:
#  ressized= resize_images(images[image])

In [0]:
#save resized images into images.
images = [resize_images(img) for img in images]

In [0]:
images = np.array(images)
classes = np.array(classes)

In [0]:
import sys 
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.callbacks import TensorBoard
from keras.applications.mobilenet import MobileNet

# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

In [0]:
images = np.array(images)
classes = np.array(classes)
# Label encoder
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
encodedLabels = enc.fit_transform(classes)

In [0]:
# Spliting data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, encodedLabels, test_size=0.2, random_state=80)

In [25]:
X_train.shape

(2443, 224, 224, 3)

In [0]:

from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import numpy as np

batch_size = 16
num_classes = 4

#base_model = resnet50.ResNet50
#base_model = inception_v3.InceptionV3
base_model = vgg16.VGG16

In [27]:
base_model = base_model(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

x_train = np.random.normal(loc=127, scale=127, size=(50, 224,224,3))
y_train = np.array([0,1]*25)
x_train = vgg16.preprocess_input(x_train)





58892288/58889256 [==============================] - 6s 0us/step










In [29]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [0]:
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='/content/drive/My Drive/Colab Notebooks/', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

In [32]:
print(model.evaluate(x_train, y_train, batch_size=batch_size, verbose=0))
model.fit(x_train, y_train,
          epochs=100,
          batch_size=batch_size,
          shuffle=False,
          validation_data=(x_train, y_train),
           callbacks=[tbCallBack])

[1.7030588483810425, 0.5]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50 samples, validate on 50 samples




Epoch 1/100
50/50 [==============================] - 1s 15ms/step - loss: 1.2043 - acc: 0.5600 - val_loss: 0.7685 - val_acc: 0.5400

Epoch 2/100
50/50 [==============================] - 1s 13ms/step - loss: 0.7060 - acc: 0.5600 - val_loss: 0.7469 - val_acc: 0.5400
Epoch 3/100
50/50 [==============================] - 1s 13ms/step - loss: 0.7022 - acc: 0.5800 - val_loss: 0.6250 - val_acc: 0.7000
Epoch 4/100
50/50 [==============================] - 1s 13ms/step - loss: 0.6272 - acc: 0.6600 - val_loss: 0.5885 - val_acc: 0.7000
Epoch 5/100
50/50 [==============================] - 1s 12ms/step - loss: 0.5950 - acc: 0.6800 - val_loss: 0.5884 - val_acc: 0.7000
Epoch 6/100
50/50 [==============================] - 1s 13ms/step - loss: 0.5727 - acc: 0.7000 - val_loss: 0.5552 - val_acc: 0.6800
Epoch 7/100
50/50 [==================